Database Project
==============================

This is our first example of an in class project.  In this project I will be going through the steps
that are listed below to help us to create a new system that can act like a database.  This will work
by using the following items. 

1. Functions
  * especially lambdas
2. Generators
3. Decorators
4. Set Operations

### Permissions

We will implement the idea of a permission for each of our database functions, we will do this by creating a 
decorator that will look at our global user and if the user has the required permission they can run it.  

    @authorize('Admin')
    def delete_query():
        pass

### Streams

We will be able to create queries that can "join" by the fact that they will be each returning a generator
(not a set list) that we can then use to create other generators.  

    def select_query(where):
        return (x for x in table if where(x))

    def join(query1, field1, query2, field2):
        return ((q1, q2) for q1 in query1 for q2 in query2 if q1[field1] == q2[field2])

## Helper functions

At this point I am going to create a few helper functions that will be used for this system. This
will help in data generation and other operations.  

In [1]:
import random



## First Steps

For the first step we are going to define our global values, this includes our tables and our user info.  

We will also want to create some test data that we will put into our created tables.  

In [2]:
# Student Table (id*, name*)
student_table = []
# Class Table (id*, name*, time)
class_table = []
# Assignment Table (id*, name*, total_score*)
assignment_table = []
# Students Classes Table (student_id*, class_id*, grade)
student_class_table = []
# Student Assignment Table (student_id*, class_id*, assignment_id*, score)
student_assignment_table = []

In [3]:
users_list = {
    'teacher': {'Admin', 'User'},
    'student': {'User'},
}

user = 'student'

In [4]:
# Student Table (id*, name*)
student_table.extend([
    (1, 'Alfred'), 
    (2, 'Barnes'),
    (3, 'Catherine'),
    (4, 'Doug'), 
    (5, 'Edward'),
    (6, 'Fonz'),
    (7, 'George'),
    (8, 'Harriet'),
    (9, 'Isaac'),
    (10, 'Justin'),
    (11, 'Kelly'),
    (12, 'Lucy'),
    (13, 'Mike'),
    (14, 'Nate'),
    (15, 'Oscar'),
    (16, 'Peter'),
    (17, 'Quincy'),
    (18, 'Rose'),
    (19, 'Sally'),
    (20, 'Terry')
])

In [5]:
# Class Table (id*, name*, time)
class_table.extend([
    (1, 'Algebra', '6:00'),
    (2, 'Biology', '3:00'),
    (3, 'Chemistry', '4:00'),
    (4, 'Dance', '5:00'),
    (5, 'English', '8:00'),
    (6, 'Finance', '1:00'),
])

In [6]:
# Assignment Table (id*, name*, total_score*)
assignment_table.extend([
    (1, 'Intro', 10),
    (2, 'Essay', 25),
    (3, 'Midterm', 50),
    (4, 'Project', 100),
    (5, 'Final', 50),
])

In [7]:
# Students Classes Table (student_id*, class_id*, grade)
student_class_table.extend(
  [(s[0], c[0], None) for s in student_table
                      for c in class_table
                      if random.randint(1, 2) == 1]
)

In [8]:
# print(student_class_table)
# Student Assignment Table (student_id*, class_id*, assignment_id*, score)
student_assignment_table.extend(
  [(sc[0], sc[1], a[0], random.randint(0, a[2])) 
     for sc in student_class_table
     for a in assignment_table]
)

#student_assignment_table.extend(
#  [(s[0], c[1], a[0], random.randint(0, a[2])) 
#     for s in student_table
#     for c in class_table
#     for a in assignment_table
#     for sc in student_class_table
#     if s[0] == sc[0] and c[0] == sc[1]]
#)

In [9]:
#print(student_assignment_table)

## Second step

At this point we are going to create a few helper functions that we can use to query data

In [10]:
def students(scope):
    if scope in users_list[user]:
        for a in student_table:
            yield a
    else:
        raise ValueError('Not authorized')

In [11]:
user = 'student'
print([a for a in students('User')])

[(1, 'Alfred'), (2, 'Barnes'), (3, 'Catherine'), (4, 'Doug'), (5, 'Edward'), (6, 'Fonz'), (7, 'George'), (8, 'Harriet'), (9, 'Isaac'), (10, 'Justin'), (11, 'Kelly'), (12, 'Lucy'), (13, 'Mike'), (14, 'Nate'), (15, 'Oscar'), (16, 'Peter'), (17, 'Quincy'), (18, 'Rose'), (19, 'Sally'), (20, 'Terry')]


In [12]:
def authorize(scope):
    def wrap(func):
        def inner(*args, **kwargs):
            if scope in users_list[user]:
                return func(*args, **kwargs)
            else:
                raise ValueError('Not authorized')
        return inner
    return wrap

In [13]:
@authorize('User')
def student():
    for a in student_table:
        yield a

In [14]:
user = 'teacher'
print([a for a in student()])

[(1, 'Alfred'), (2, 'Barnes'), (3, 'Catherine'), (4, 'Doug'), (5, 'Edward'), (6, 'Fonz'), (7, 'George'), (8, 'Harriet'), (9, 'Isaac'), (10, 'Justin'), (11, 'Kelly'), (12, 'Lucy'), (13, 'Mike'), (14, 'Nate'), (15, 'Oscar'), (16, 'Peter'), (17, 'Quincy'), (18, 'Rose'), (19, 'Sally'), (20, 'Terry')]


In [15]:
def bad_guy(a, b):
    def wrapper(func):
        def inner(*args, **kwargs):
            return func(a, b)
        return inner
    return wrapper

@bad_guy(4, 5)
def my_sum(a, b):
    return a + b

print(my_sum(1, 2))
print(my_sum(3, 4))
print(my_sum(0, 0))

9
9
9


In [16]:
@authorize('User')
def classes():
    for c in class_table:
        yield c
        
@authorize('User')
def assignment():
    for a in assignment_table:
        yield a
        
@authorize('Admin')
def student_class():
    for sc in student_class_table:
        yield sc
        
@authorize('Admin')
def student_assignment():
    for sca in student_assignment_table:
        yield sca

In [17]:
user = 'teacher'
print([a for a in student_class()])

[(1, 1, None), (1, 2, None), (1, 4, None), (1, 5, None), (1, 6, None), (3, 1, None), (3, 2, None), (3, 3, None), (3, 4, None), (3, 5, None), (3, 6, None), (4, 3, None), (5, 1, None), (5, 3, None), (5, 5, None), (5, 6, None), (6, 3, None), (6, 4, None), (6, 5, None), (7, 1, None), (8, 3, None), (8, 4, None), (8, 5, None), (8, 6, None), (9, 2, None), (9, 3, None), (9, 4, None), (9, 5, None), (10, 1, None), (10, 2, None), (10, 3, None), (10, 4, None), (10, 5, None), (11, 1, None), (11, 4, None), (12, 3, None), (12, 6, None), (13, 3, None), (13, 6, None), (14, 1, None), (14, 2, None), (14, 3, None), (14, 4, None), (14, 5, None), (14, 6, None), (15, 1, None), (15, 2, None), (16, 1, None), (16, 2, None), (16, 4, None), (16, 6, None), (17, 2, None), (17, 4, None), (17, 5, None), (18, 1, None), (18, 3, None), (18, 5, None), (18, 6, None), (19, 1, None), (19, 2, None), (19, 4, None), (19, 6, None), (20, 3, None), (20, 4, None), (20, 5, None), (20, 6, None)]


## Queries

* Student names who are in Dance or Finance.  
* Students in Dance and Chemistry.
* Sum of scores above 50% for Essay assignment in class Chemistry
* Get a tuple of all the students that are in the different classes


In [18]:
results = {s[1] for s in student()
                for c in classes()
                for sc in student_class()
                if c[1] == 'Dance' or c[1] == 'Finance'
                if s[0] == sc[0]
                if c[0] == sc[1]
          }
print(results)

{'Fonz', 'Mike', 'Harriet', 'Rose', 'Alfred', 'Terry', 'Justin', 'Edward', 'Quincy', 'Lucy', 'Isaac', 'Nate', 'Kelly', 'Peter', 'Sally', 'Catherine'}


In [19]:
def student_in_class(class_name):
    return {s[1] for s in student()
                 for c in classes()
                 for sc in student_class()
                 if c[1] == class_name
                 if s[0] == sc[0]
                 if c[0] == sc[1]
            }

dance_result = student_in_class('Dance')
chemistry_result = student_in_class('Chemistry')
#result = { s for s in dance_result for c in chemistry_result if c == s}
result = dance_result & chemistry_result
print(result)

{'Fonz', 'Harriet', 'Terry', 'Justin', 'Nate', 'Isaac', 'Catherine'}


In [21]:
results = [sa[3] for sa in student_assignment()
              for c in classes()
              for a in assignment()
              if c[1] == 'Chemistry'
              if sa[1] == c[0]
              if a[1] == 'Essay'
              if sa[2] == a[0]
              if sa[3] >= a[2] / 2]

print(results)
print(sum(results))

[17, 13, 13, 15, 14, 23]
95


In [23]:
class_list = [c for c in classes()]
results = []
for c in class_list:
    results.append(tuple({ s[1] for s in student()
                                for sc in student_class()
                                if sc[1] == c[0]
                                if sc[0] == s[0]}))
print(results)

[('Oscar', 'Rose', 'Alfred', 'George', 'Justin', 'Edward', 'Nate', 'Kelly', 'Peter', 'Sally', 'Catherine'), ('Oscar', 'Alfred', 'Justin', 'Quincy', 'Isaac', 'Nate', 'Peter', 'Sally', 'Catherine'), ('Fonz', 'Mike', 'Harriet', 'Rose', 'Terry', 'Justin', 'Edward', 'Doug', 'Lucy', 'Isaac', 'Nate', 'Catherine'), ('Fonz', 'Harriet', 'Terry', 'Alfred', 'Justin', 'Quincy', 'Isaac', 'Nate', 'Kelly', 'Peter', 'Sally', 'Catherine'), ('Fonz', 'Harriet', 'Rose', 'Alfred', 'Terry', 'Justin', 'Edward', 'Quincy', 'Isaac', 'Nate', 'Catherine'), ('Mike', 'Harriet', 'Rose', 'Alfred', 'Terry', 'Edward', 'Lucy', 'Nate', 'Peter', 'Sally', 'Catherine')]
